In [1]:
%matplotlib inline
from textblob import TextBlob, Word
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

In [3]:
import csv
import pandas as pd
with open('reviews_suomi.txt', encoding='utf-8') as f:
  result = f.readlines()

#translated = GoogleTranslator(source='auto', target='en').translate(str(result))

In [4]:
import re
result_clean = re.sub(r'\W+', ' ', str(result))

In [5]:
type(result_clean)

str

In [6]:
# Perform standard imports
import spacy
from spacy.matcher import PhraseMatcher #import PhraseMatcher class
nlp = spacy.load('spacy_fi_experimental_web_md')

In [7]:
result_clean_nlp = nlp(result_clean)
lem_list=[]
for t in result_clean_nlp:
    lem_list.append(t.lemma_)

In [8]:
lem_list

[' ',
 'ei',
 'tarkastella',
 'ku',
 '30min',
 'ja',
 'sit',
 'pitää',
 'sallia',
 'uudestaan',
 'et',
 'saada',
 'tausta',
 'toimi',
 'eli',
 'iha',
 'p',
 'sovellus',
 'Muutenkii',
 'koko',
 'korona',
 'hössötys',
 'olla',
 'iha',
 'turha',
 'ei',
 'haluis',
 'antaa',
 'yhtäkää',
 'tähti',
 'n',
 'lähipiiri',
 'koronatartunta',
 'ei',
 'näkyä',
 'sovellus',
 'ollenkaan',
 'tuntikausi',
 'jo',
 'mennä',
 'puhelimitse',
 'ilmoitettu',
 'tartunta',
 'kaveri',
 'olla',
 'n',
 '5',
 '15',
 'm',
 'päässä',
 'meikäläinen',
 'yli',
 'tunti',
 'n',
 'kun',
 'lukea',
 'arvostelu',
 'kokemus',
 'niin',
 'ei',
 'kyllä',
 'voida',
 'luottaa',
 'koronavilkku',
 'yhtään',
 'ei',
 'taitaa',
 'olla',
 'mikään',
 'hyöty',
 'n',
 'ei',
 'toimia',
 'päivitys',
 'jälkeen',
 'hypätä',
 'pois',
 'koko',
 'sovellus',
 'kun',
 'se',
 'yrittää',
 'avata',
 'n',
 'ei',
 'päivittyä',
 'automaattisesti',
 'joutua',
 'käynnistää',
 'puhelin',
 'uudestaan',
 'kun',
 'sitten',
 'joskus',
 'muistaa',
 'tarkastaa',
 

'  ei tarkastella ku 30min ja sit pitää sallia uudestaan et saada tausta toimi eli iha p sovellus Muutenkii koko korona hössötys olla iha turha ei haluis antaa yhtäkää tähti n lähipiiri koronatartunta ei näkyä sovellus ollenkaan tuntikausi jo mennä puhelimitse ilmoitettu tartunta kaveri olla n 5 15 m päässä meikäläinen yli tunti n kun lukea arvostelu kokemus niin ei kyllä voida luottaa koronavilkku yhtään ei taitaa olla mikään hyöty n ei toimia päivitys jälkeen hypätä pois koko sovellus kun se yrittää avata n ei päivittyä automaattisesti joutua käynnistää puhelin uudestaan kun sitten joskus muistaa tarkastaa tilanne tänään 29 10 ja juuri äsken tarkastaa viimeksi 24 10 puhelin Huawei n ei toimia ilman GPS paikannus vaikka niin kovin väittää Adroid9 ei näköjään Andr10 skään n sovellus ei päivittyä automaattisesti ei asetus löytyä päivitys pitää mennä Google Playihin ja sieltä asentaa toimia tuo koko sovellus Nokia 7 2 Android n ei päivittyä viimeinen tarkistus kolme päivä sitten puhelin 

In [12]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words("finnish")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Men\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
tokens_without_sw = [word for word in lem_list if not word in stopwords]

In [17]:
tokens_without_sw

[' ',
 'tarkastella',
 'ku',
 '30min',
 'sit',
 'pitää',
 'sallia',
 'uudestaan',
 'saada',
 'tausta',
 'toimi',
 'eli',
 'iha',
 'p',
 'sovellus',
 'Muutenkii',
 'koko',
 'korona',
 'hössötys',
 'iha',
 'turha',
 'haluis',
 'antaa',
 'yhtäkää',
 'tähti',
 'n',
 'lähipiiri',
 'koronatartunta',
 'näkyä',
 'sovellus',
 'ollenkaan',
 'tuntikausi',
 'jo',
 'mennä',
 'puhelimitse',
 'ilmoitettu',
 'tartunta',
 'kaveri',
 'n',
 '5',
 '15',
 'm',
 'päässä',
 'meikäläinen',
 'tunti',
 'n',
 'lukea',
 'arvostelu',
 'kokemus',
 'kyllä',
 'voida',
 'luottaa',
 'koronavilkku',
 'yhtään',
 'taitaa',
 'mikään',
 'hyöty',
 'n',
 'toimia',
 'päivitys',
 'jälkeen',
 'hypätä',
 'pois',
 'koko',
 'sovellus',
 'yrittää',
 'avata',
 'n',
 'päivittyä',
 'automaattisesti',
 'joutua',
 'käynnistää',
 'puhelin',
 'uudestaan',
 'sitten',
 'joskus',
 'muistaa',
 'tarkastaa',
 'tilanne',
 'tänään',
 '29',
 '10',
 'juuri',
 'äsken',
 'tarkastaa',
 'viimeksi',
 '24',
 '10',
 'puhelin',
 'Huawei',
 'n',
 'toimia',
 

In [18]:
stringList = ' '.join([str(item) for item in tokens_without_sw])

In [19]:
stringList

'  tarkastella ku 30min sit pitää sallia uudestaan saada tausta toimi eli iha p sovellus Muutenkii koko korona hössötys iha turha haluis antaa yhtäkää tähti n lähipiiri koronatartunta näkyä sovellus ollenkaan tuntikausi jo mennä puhelimitse ilmoitettu tartunta kaveri n 5 15 m päässä meikäläinen tunti n lukea arvostelu kokemus kyllä voida luottaa koronavilkku yhtään taitaa mikään hyöty n toimia päivitys jälkeen hypätä pois koko sovellus yrittää avata n päivittyä automaattisesti joutua käynnistää puhelin uudestaan sitten joskus muistaa tarkastaa tilanne tänään 29 10 juuri äsken tarkastaa viimeksi 24 10 puhelin Huawei n toimia ilman GPS paikannus kovin väittää Adroid9 näköjään Andr10 skään n sovellus päivittyä automaattisesti asetus löytyä päivitys pitää mennä Google Playihin sieltä asentaa toimia koko sovellus Nokia 7 2 Android n päivittyä viimeinen tarkistus kolme päivä sitten puhelin Huawei n mikään ilmoitus lähellä tartunta usea altistaa n ilmoittaa mikään alku muuttaa viikko väli sit

In [22]:
wordlist = stringList.split()

wordfreq = []
for w in wordlist:
    wordfreq.append(wordlist.count(w))

print("Pairs\n" + str(list(zip(wordlist, wordfreq))))

Pairs
[('tarkastella', 2), ('ku', 7), ('30min', 2), ('sit', 3), ('pitää', 142), ('sallia', 2), ('uudestaan', 30), ('saada', 119), ('tausta', 22), ('toimi', 6), ('eli', 29), ('iha', 6), ('p', 4), ('sovellus', 441), ('Muutenkii', 2), ('koko', 52), ('korona', 46), ('hössötys', 2), ('iha', 6), ('turha', 32), ('haluis', 2), ('antaa', 27), ('yhtäkää', 2), ('tähti', 15), ('n', 1007), ('lähipiiri', 2), ('koronatartunta', 2), ('näkyä', 33), ('sovellus', 441), ('ollenkaan', 11), ('tuntikausi', 2), ('jo', 30), ('mennä', 26), ('puhelimitse', 2), ('ilmoitettu', 2), ('tartunta', 67), ('kaveri', 2), ('n', 1007), ('5', 30), ('15', 11), ('m', 6), ('päässä', 6), ('meikäläinen', 2), ('tunti', 14), ('n', 1007), ('lukea', 15), ('arvostelu', 9), ('kokemus', 8), ('kyllä', 31), ('voida', 92), ('luottaa', 6), ('koronavilkku', 56), ('yhtään', 14), ('taitaa', 10), ('mikään', 70), ('hyöty', 19), ('n', 1007), ('toimia', 285), ('päivitys', 43), ('jälkeen', 89), ('hypätä', 2), ('pois', 52), ('koko', 52), ('sovellus'

In [27]:
# Given a list of words, return a dictionary of
# word-frequency pairs.
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))

In [32]:
# Sort a dictionary of word-frequency pairs in
# order of descending frequency.
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [33]:

dictionary = wordListToFreqDict(tokens_without_sw)
sorteddict = sortFreqDict(dictionary)

for s in sorteddict: print(str(s))

(1007, 'n')
(441, 'sovellus')
(285, 'toimia')
(222, 'puhelin')
(145, 'hyvä')
(142, 'pitää')
(125, 'tehdä')
(119, 'saada')
(104, 'akku')
(94, 'päällä')
(93, 'tarkistaa')
(92, 'voida')
(92, 'asentaa')
(91, 'altistuminen')
(89, 'jälkeen')
(86, 'päivä')
(85, 'tarkistus')
(83, 'uudelleen')
(83, 'päivittää')
(83, 'käyttö')
(75, 'tulla')
(73, 'helppo')
(71, 'aika')
(70, 'mikään')
(70, 'kaikki')
(68, '9')
(67, 'tartunta')
(67, 'sitten')
(66, 'tieto')
(60, 'muuten')
(60, 'ilmoitus')
(59, 'käyttää')
(58, 'ilmoittaa')
(57, 'ladata')
(56, 'koronavilkku')
(54, '10')
(53, 'vain')
(52, 'poistaa')
(52, 'pois')
(52, 'koko')
(48, 'Huawei')
(47, 'käynnistää')
(46, 'päivittyä')
(46, 'korona')
(45, 'myös')
(44, 'Bluetooth')
(43, 'päivitys')
(43, 'aina')
(42, 'tietää')
(42, 'muu')
(42, 'ihan')
(41, 'viimeksi')
(40, 'ottaa')
(39, 'laite')
(37, 'vilkku')
(37, 'kerran')
(37, 'asennus')
(35, 'kerta')
(35, 'bluetooth')
(35, 'auttaa')
(34, 'sijainti')
(34, 'ihminen')
(34, 'hyvin')
(33, 'näkyä')
(33, 'asetus')
(33

(1, 'julkinen')
(1, 'joutaa')
(1, 'jottei')
(1, 'jonne')
(1, 'jonku')
(1, 'jollekkin')
(1, 'jollainlailla')
(1, 'jiono')
(1, 'jees')
(1, 'isoveli')
(1, 'informoida')
(1, 'ilmotusta')
(1, 'ilmoitusalue')
(1, 'ilmiselvästi')
(1, 'ilmaista')
(1, 'ilma')
(1, 'ikoni')
(1, 'ikinä')
(1, 'ihme')
(1, 'höösäämisellä')
(1, 'höpö')
(1, 'hämäräperäinen')
(1, 'häkkeröidä')
(1, 'hyödyntää')
(1, 'hyvästi')
(1, 'hyljätä')
(1, 'huoli')
(1, 'hukkaan')
(1, 'housuun')
(1, 'homma')
(1, 'hirveä')
(1, 'hiljainen')
(1, 'hemmetisti')
(1, 'havaittu')
(1, 'havainto')
(1, 'harvoin')
(1, 'good')
(1, 'gates')
(1, 'far')
(1, 'evästie')
(1, 'esimerkiksi')
(1, 'ero')
(1, 'erityisesti')
(1, 'epäselvä')
(1, 'epäaktiivinen')
(1, 'ensivaikutelma')
(1, 'ensin')
(1, 'enempää')
(1, 'edge')
(1, 'edellisenkerta')
(1, 'boottausta')
(1, 'bongata')
(1, 'bill')
(1, 'avauskoodi')
(1, 'auttanu')
(1, 'asuntola')
(1, 'asunola')
(1, 'asiakas')
(1, 'asennusajankohta')
(1, 'asennin')
(1, 'asennella')
(1, 'ase')
(1, 'arviointi')
(1, 'arka'

In [44]:
import logging
#from stop_words import get_stop_words
from gensim import corpora,models
import gensim
import os, operator
from collections import defaultdict

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def topics_prct(lda, topic_count, word_count, corpus):
    log = logging.getLogger("bob") #the logger seems to require some id so i just call it "bob"
    topic_words = {}
    for t in range(topic_count):
        # top is now list of tuples (word, probability). topn=number of words to take
        top = lda.show_topic(t, topn=word_count)
        topic_words[t] = top

    topic_sizes = defaultdict(int)

    for doc in corpus:
        #doc_box is the document represented as a bag of words
        doc_bow = doc
        dist = lda[doc_bow]
        for topic_word in dist:
            # count topic sizes by summing the probabilities of all words in all docs to be in that topic
            # so the topic_word is in topic_id and it makes up "percent" of words in the document (instances of one word can be in different topics across the doc)
            topic_id = topic_word[0]
            percent = topic_word[1]
            topic_sizes[topic_id] += percent
    print("sized topics")

    topic_words_weighted = {}
    for t in range(topic_count):
        t_words = topic_words[t] #the top words in topic "t"
        topic_size = topic_sizes[t] #the total size of the topic "t", sum of its word probabilities
        tw_words = []
        topic_words_weighted[t] = tw_words
        for word, percent in t_words:
            my_tuple = (word, percent * topic_size) #build a comparable list of words/topic to represent their sizes. not exactly the number of appearances since I could not figure that for gensim, but serves the purpose
            tw_words.append(my_tuple)

    log.info("sized words")

    #sort the topics by their ID values, so if we print the list it will be in numerical order. could also sort by the topic size (item 1 I guess) to get biggest topics first
    sorted_topics = sorted(topic_sizes.items(), key=operator.itemgetter(0))

    file_data = ""

    for topic in sorted_topics:
        topic_id = topic[0]
        file_data += "topic"+str(topic_id)+"="
        tww = topic_words_weighted[topic_id]
        for tw in tww:
            word_size = int(tw[1]*100)
            file_data += tw[0]+"["+str(word_size)+"] "
        file_data += "\n"

    log.info("built file data")
    print(file_data)
    return file_data

def create_model(topic_count, doc_limit, iterations):
    stoplist = stopwords

    documents = []

    print("starting to read docs")
    count = 0
    #i put all the preprocessed (lemmatized, known cruft and stopwords removed) into "wikidump" dir, this would differ for different datasets, or you might just directly read from some source and preprocess here
    for filename in os.listdir("wikidump"):
        with open("wikidump/"+filename, "r") as file:
            data = file.read()
            #drop single letter words and words in the list of stop words, also lowercase all
            #i guess this might have an issue of removing stopwords in different upper/lowercase combos but but..
			#also, i did preprocess the docs before but good opportunity to fix here what was missed in preprocessing..
            doc = [word for word in data.lower().split() if (word not in stoplist and len(word) > 2)]
            documents.append(doc)
            count += 1
            if count > doc_limit:
                break

    print("finished reading docs")
    print("got "+str(count)+" docs. starting model build.")

    #this is just basic application of gensim according to gensim docs/tutorials
    dictionary = corpora.Dictionary(documents)
    print("dictionary built")
    corpus = [dictionary.doc2bow(text) for text in documents]
    print("corpus built")
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topic_count, id2word = dictionary, passes=iterations)
    print("lda built")

    print(ldamodel.print_topics(num_topics=topic_count, num_words=10))
    return ldamodel, corpus

def write_file(txt):
    topics_file = open("topics.txt", "w")
    topics_file.write(txt)
    topics_file.close()

topic_count = 50
word_count = 10
doc_limit = 50000000 #you can use the doc_limit here to cut the used dataset to use only first N documents. useful to quickly test stuff
iterations = 2
lda_model, corpus = create_model(topic_count, doc_limit, iterations)
topics_txt = topics_prct(lda_model, topic_count, word_count, corpus)
write_file(topics_txt)


2021-03-19 11:02:22,769 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-03-19 11:02:22,780 : INFO : built Dictionary(1749 unique tokens: ['100', '14d', '14pv', '15min', '19pã¤ivã¤n']...) from 1 documents (total 11483 corpus positions)
2021-03-19 11:02:22,788 : INFO : using symmetric alpha at 0.02
2021-03-19 11:02:22,789 : INFO : using symmetric eta at 0.02
2021-03-19 11:02:22,791 : INFO : using serial LDA version on this node
2021-03-19 11:02:22,808 : INFO : running online (multi-pass) LDA training, 50 topics, 2 passes over the supplied corpus of 1 documents, updating model once every 1 documents, evaluating perplexity every 1 documents, iterating 50x with a convergence threshold of 0.001000
2021-03-19 11:02:22,809 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2021-03-19 11:02:22,852 : INFO : -26.644 per-word bound, 104875381.4 perplexity estimate based on a held-out corpus of 1 d

starting to read docs
finished reading docs
got 1 docs. starting model build.
dictionary built
corpus built


2021-03-19 11:02:22,880 : INFO : topic #6 (0.020): 0.052*"sovellus" + 0.019*"toimia" + 0.017*"puhelin" + 0.014*"saada" + 0.013*"hyvã¤" + 0.013*"pitã¤ã¤" + 0.011*"akku" + 0.011*"tehdã¤" + 0.008*"pã¤ivittã¤ã¤" + 0.008*"pã¤ã¤llã¤"
2021-03-19 11:02:22,883 : INFO : topic #5 (0.020): 0.044*"sovellus" + 0.022*"toimia" + 0.018*"puhelin" + 0.013*"saada" + 0.011*"pitã¤ã¤" + 0.011*"hyvã¤" + 0.011*"tehdã¤" + 0.008*"pã¤ã¤llã¤" + 0.008*"asentaa" + 0.008*"uudelleen"
2021-03-19 11:02:22,884 : INFO : topic #36 (0.020): 0.032*"sovellus" + 0.023*"toimia" + 0.020*"puhelin" + 0.012*"hyvã¤" + 0.011*"tehdã¤" + 0.010*"pitã¤ã¤" + 0.010*"altistuminen" + 0.009*"voida" + 0.009*"pã¤ivã¤" + 0.009*"tarkistaa"
2021-03-19 11:02:22,886 : INFO : topic #30 (0.020): 0.041*"sovellus" + 0.020*"toimia" + 0.019*"puhelin" + 0.013*"pitã¤ã¤" + 0.010*"hyvã¤" + 0.010*"saada" + 0.010*"tarkistaa" + 0.009*"akku" + 0.009*"tehdã¤" + 0.008*"jã¤lkeen"
2021-03-19 11:02:22,887 : INFO : topic #47 (0.020): 0.026*"sovellus" + 0.023*"toimia" +

2021-03-19 11:02:23,071 : INFO : topic #25 (0.020): 0.025*"sovellus" + 0.019*"toimia" + 0.014*"puhelin" + 0.010*"pitã¤ã¤" + 0.009*"tehdã¤" + 0.008*"saada" + 0.008*"hyvã¤" + 0.008*"akku" + 0.006*"voida" + 0.006*"uudelleen"
2021-03-19 11:02:23,073 : INFO : topic #26 (0.020): 0.012*"sovellus" + 0.010*"toimia" + 0.010*"puhelin" + 0.006*"pitã¤ã¤" + 0.005*"hyvã¤" + 0.005*"voida" + 0.004*"tehdã¤" + 0.004*"asentaa" + 0.004*"akku" + 0.004*"saada"
2021-03-19 11:02:23,074 : INFO : topic #27 (0.020): 0.024*"sovellus" + 0.019*"toimia" + 0.013*"puhelin" + 0.011*"pitã¤ã¤" + 0.010*"akku" + 0.010*"tehdã¤" + 0.008*"hyvã¤" + 0.007*"saada" + 0.007*"voida" + 0.007*"bluetooth"
2021-03-19 11:02:23,075 : INFO : topic #28 (0.020): 0.022*"sovellus" + 0.014*"toimia" + 0.013*"puhelin" + 0.010*"hyvã¤" + 0.010*"pitã¤ã¤" + 0.007*"asentaa" + 0.007*"jã¤lkeen" + 0.007*"voida" + 0.007*"saada" + 0.006*"tehdã¤"
2021-03-19 11:02:23,077 : INFO : topic #29 (0.020): 0.014*"sovellus" + 0.011*"toimia" + 0.008*"puhelin" + 0.006*

lda built
[(0, '0.029*"sovellus" + 0.017*"puhelin" + 0.016*"toimia" + 0.011*"pitã¤ã¤" + 0.009*"hyvã¤" + 0.008*"saada" + 0.008*"voida" + 0.007*"pã¤ivã¤" + 0.007*"jã¤lkeen" + 0.006*"akku"'), (1, '0.023*"sovellus" + 0.013*"toimia" + 0.009*"hyvã¤" + 0.009*"puhelin" + 0.006*"saada" + 0.006*"asentaa" + 0.006*"tehdã¤" + 0.005*"pitã¤ã¤" + 0.005*"akku" + 0.005*"pã¤ã¤llã¤"'), (2, '0.014*"sovellus" + 0.012*"toimia" + 0.009*"puhelin" + 0.006*"tehdã¤" + 0.005*"tarkistaa" + 0.005*"akku" + 0.005*"asentaa" + 0.005*"pã¤ã¤llã¤" + 0.005*"pitã¤ã¤" + 0.005*"saada"'), (3, '0.037*"sovellus" + 0.018*"toimia" + 0.017*"puhelin" + 0.013*"pitã¤ã¤" + 0.010*"tehdã¤" + 0.009*"saada" + 0.008*"akku" + 0.008*"hyvã¤" + 0.007*"pã¤ã¤llã¤" + 0.007*"tarkistaa"'), (4, '0.017*"sovellus" + 0.011*"toimia" + 0.010*"puhelin" + 0.008*"hyvã¤" + 0.006*"akku" + 0.005*"asentaa" + 0.005*"saada" + 0.005*"altistuminen" + 0.005*"tulla" + 0.005*"tehdã¤"'), (5, '0.040*"sovellus" + 0.020*"toimia" + 0.017*"puhelin" + 0.012*"saada" + 0.010*"pi